# Import packages

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
!pip install transformers 
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.2 MB/s 
     |████████████████████████████████| 163 kB 87.0 MB/s 
     |████████████████████████████████| 7.6 MB 79.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 432 kB 5.0 MB/s 
     |████████████████████████████████| 115 kB 69.8 MB/s 
     |████████████████████████████████| 212 kB 71.5 MB/s 
     |████████████████████████████████| 127 kB 78.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import torch
import pandas as pd
import numpy as np
from transformers import RobertaTokenizer, AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, RobertaForSequenceClassification
from datasets import load_dataset, ClassLabel, Value, load_metric

# load source dataset

In [ ]:
# sampling the dataset for fine-tuning
train = load_dataset('amazon_us_reviews', 'Video_v1_00', split='train[:60%]') 

Generating train split:   0%|          | 0/380604 [00:00<?, ? examples/s]

Dataset amazon_us_reviews downloaded and prepared to /root/.cache/huggingface/datasets/amazon_us_reviews/Video_v1_00/0.1.0/17b2481be59723469538adeb8fd0a68b0ba363bbbdd71090e72c325ee6c7e563. Subsequent calls will reuse this data.


In [ ]:
# take a look at a sample
train[0]

{'marketplace': 'US',
 'customer_id': '49033728',
 'review_id': 'R1P1G5KZ05H6RD',
 'product_id': '6302503213',
 'product_parent': '748506413',
 'product_title': 'The Night They Saved Christmas [VHS]',
 'product_category': 'Video',
 'star_rating': 5,
 'helpful_votes': 0,
 'total_votes': 0,
 'vine': 0,
 'verified_purchase': 1,
 'review_headline': 'Very satisfied!!',
 'review_body': 'Fast shipping. Pleasure to deal with. Would recommend. A+++. Thanks!',
 'review_date': '2015-08-31'}

# preprocessing

In [ ]:
# remove unuseful columns
train = train.remove_columns(['marketplace', 'review_id', 'product_parent', 'product_title', 'product_category', \
                      'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_date'])

In [ ]:
train[0]

{'customer_id': '49033728',
 'product_id': '6302503213',
 'star_rating': 5,
 'review_headline': 'Very satisfied!!',
 'review_body': 'Fast shipping. Pleasure to deal with. Would recommend. A+++. Thanks!'}

# Encoding 

In [ ]:
# load the tokenizer pretrained on roberta-base
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
# encode the training dataset in the form of sentences pair
# truncate at length=32 for a balance of time consuming and information coverage
train_tokenized = train.map(lambda batch: tokenizer(batch['review_body'], padding='max_length', truncation=True, max_length=64))

  0%|          | 0/228362 [00:00<?, ?ex/s]

In [ ]:
train_tokenized = train_tokenized.rename_column("star_rating", "labels")
train_tokenized = train_tokenized.rename_column("review_body", "text")

In [ ]:
# convert star rating that ranging from 1-5 to labels that ranging from 0-4
def to_label(x):
    x['labels']  = x['labels'] - 1
    return x

train_tokenized = train_tokenized.map(to_label)

  0%|          | 0/228362 [00:00<?, ?ex/s]

In [ ]:
train_tokenized.set_format("torch", columns=["input_ids",  "attention_mask", "labels"])

In [ ]:
train_tokenized[0]

{'labels': tensor(4),
 'input_ids': tensor([    0, 35515,  6738,     4, 18486, 24669,     7,   432,    19,     4,
         11258,  5940,     4,    83, 42964, 27079,  4557,   328,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

# Fine-tuning

In [ ]:
# load a pretrained model

model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=5)

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [ ]:
# set training arguments manually if needed, otherwise use the defalut
training_args = TrainingArguments(
    output_dir='./output',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=0,                # number of warmup steps for learning rate scheduler
    learning_rate=5e-5,               # learning rate
    logging_dir='./logs',            # directory for storing logs
    logging_steps=1000,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
metric = load_metric("accuracy")

In [ ]:
# Create a Trainer object with the model, training arguments, training and test datasets, and evaluation function
trainer = Trainer(
    model=model,
    args = training_args,
    train_dataset=train_tokenized)

In [ ]:
# clean up gpu cache before training
import gc

gc.collect()

torch.cuda.empty_cache()

In [ ]:
trainer.train()    
# trainer.train(resume_from_checkpoint=True) # True if already trained, to save time by continuing on a checkpoint

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, review_headline, product_id, customer_id. If text, review_headline, product_id, customer_id are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 228362
  Num Epochs = 5
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 17845


Step,Training Loss
1000,0.890800
2000,0.836800
3000,0.820200
4000,0.784300
5000,0.749400
6000,0.748000
7000,0.739800
8000,0.676300
9000,0.673300
10000,0.660500


Saving model checkpoint to ./output/checkpoint-500
Configuration saved in ./output/checkpoint-500/config.json
Model weights saved in ./output/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./output/checkpoint-1000
Configuration saved in ./output/checkpoint-1000/config.json
Model weights saved in ./output/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./output/checkpoint-1500
Configuration saved in ./output/checkpoint-1500/config.json
Model weights saved in ./output/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./output/checkpoint-2000
Configuration saved in ./output/checkpoint-2000/config.json
Model weights saved in ./output/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./output/checkpoint-2500
Configuration saved in ./output/checkpoint-2500/config.json
Model weights saved in ./output/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to ./output/checkpoint-3000
Configuration saved in ./output/checkpoint-3000/config.json
M

TrainOutput(global_step=17845, training_loss=0.6653989886462638, metrics={'train_runtime': 4826.9357, 'train_samples_per_second': 236.55, 'train_steps_per_second': 3.697, 'total_flos': 3.755386578085248e+16, 'train_loss': 0.6653989886462638, 'epoch': 5.0})

In [ ]:
# save the fine_tuned model

model.save_pretrained("Roberta-senti")

Configuration saved in Roberta-senti/config.json
Model weights saved in Roberta-senti/pytorch_model.bin


# Predict on the whole dataset with the fine-tuned model

In [ ]:
# load the entire dataset
dataset = load_dataset('amazon_us_reviews', 'Video_v1_00', split='train')

# remove unuseful columns
dataset = dataset.remove_columns(['marketplace', 'review_id', 'product_parent', 'product_title', 'product_category', \
                      'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_date'])
    
# load the fine-tuned tokenizer  
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# encode the training dataset in the form of sentences pair
# truncate at length=32 for a balance of time consuming and information coverage
dataset_tokenized = dataset.map(lambda batch: tokenizer(batch['review_body'], padding='max_length', truncation=True, max_length=64))
dataset_tokenized = dataset_tokenized.rename_column("star_rating", "labels")
dataset_tokenized = dataset_tokenized.rename_column("review_body", "text")

# convert star rating that ranging from 1-5 to labels that ranging from 0-4
dataset_tokenized = dataset_tokenized.map(to_label)

dataset_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_

  0%|          | 0/380604 [00:00<?, ?ex/s]

  0%|          | 0/380604 [00:00<?, ?ex/s]

In [ ]:
pred_output = trainer.predict(dataset_tokenized)

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, review_headline, product_id, customer_id. If text, review_headline, product_id, customer_id are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 380604
  Batch size = 64


In [ ]:
# get prediction and evaluate
pred = pred_output[0].argmax(axis=1)
truth = pred_output[1]
accuracy = load_metric('accuracy')
f1 = load_metric('f1')
accuracy.compute(predictions=pred, references=truth)

{'accuracy': 0.7869123813727654}

In [ ]:
f1.compute(predictions=pred, references=truth, average='weighted')

{'f1': 0.779370249430155}

# Save the dataset expanded by rating predicted by sentiment analysis

In [ ]:
# expand the original dataset with the predicted rating
data = dataset[:]
data['senti_rating_finetune'] = pred + 1

In [ ]:
df = pd.DataFrame.from_dict(data)

# extract variables needed for CF recommender
df= df[['customer_id', 'product_id', 'star_rating', 'senti_rating_finetune']]
df.rename(columns = {'customer_id' : 'user', 'product_id' : 'item', 'star_rating' : 'rating'}, inplace = True)
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,user,item,rating,senti_rating_finetune
0,49033728,6302503213,5,5
1,17857748,B000059PET,5,5
2,25551507,0788812807,4,5
3,21025041,6302509939,5,5
4,40943563,B00JENS2BI,3,3


In [ ]:
# save as csv files
df.to_csv('../amazon_video_roberta.csv')